In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold

In [9]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [3]:
import cv2
from PIL import Image
import tensorflow as tf
from random import randrange
import pickle
import json
import tensorflow.keras as keras
import os
import sys
import time

In [4]:
def load_shuffled_data():
    """
    Load dataset from pickle file and split features and labels
    returns (X_train, X_test, y_train, y_test)
    """
    start_time = time.time()

    with open("sample_train_img.pkl", 'rb') as f_name:
        train_img = pickle.load(f_name)
        train_img /= 255.0

    with open("sample_train_label.pkl", 'rb') as f_name:
        train_label = pickle.load(f_name)
        train_label = tf.keras.utils.to_categorical(train_label.astype('int'))
        
    
    with open("sample_test_img.pkl", 'rb') as f_name:
        test_img = pickle.load(f_name)
        test_img /= 255.0

    with open("sample_test_label.pkl", 'rb') as f_name:
        test_label = pickle.load(f_name)
        test_label = tf.keras.utils.to_categorical(test_label.astype('int'))

    #Shuffling Images!

    permute_train = np.random.permutation(len(train_label))
    permute_test = np.random.permutation(len(test_label))

    train_img = train_img[permute_train]
    train_label = train_label[permute_train]

    test_img = train_img[permute_train]
    test_label = train_label[permute_train]

    return(train_img, test_img, train_label, test_label)

In [5]:
def CNN_model(image_height = 256, image_width = 256, image_channel=1, class_count = 2):
    
    model = Sequential()
    #Layer 1
    model.add(Conv2D(   filters = 96, 
                        kernel_size=(11,11), 
                        strides = 4, 
                        activation='relu', 
                        kernel_initializer='he_uniform', 
                        padding = "same",
                        input_shape = (image_height, image_width, image_channel)))

    model.add(BatchNormalization())

    #Taking the maximum in that subregion using MaxPooling
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = 2))
    #he_uniform: Draw samples from uniform distributions within range of limits created with units


    #Layer 2
    model.add(Conv2D(   filters = 256, 
                        kernel_size=(5,5), 
                        strides = 1,
                        activation='relu', 
                        kernel_initializer='he_uniform'))

    model.add(BatchNormalization())
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = 2))

    #Layer 3: 
    model.add(Conv2D(   filters = 128, 
                        kernel_size=(3,3), 
                        strides = 1, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))

    model.add(BatchNormalization())
    model.add(MaxPool2D(    pool_size=(3,3),
                            strides = 2))
    
    #Layer 4
    model.add(Flatten())

    #Layer 5:
    model.add(Dense(    units = 2048, 
                        activation='relu',              
                        kernel_initializer='he_uniform'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))   

    #Layer 6
    model.add(Dense(    units = 2048, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))   

    #Layer 7
    model.add(Dense(    units = 2, 
                        activation='softmax'))

    #Compiling:
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(  optimizer=opt, 
                    loss='categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [6]:
cnn_model = CNN_model()
train_x, test_x, train_y, test_y = load_shuffled_data()

In [7]:
np.shape(train_x)

(180, 256, 256, 1)

In [ ]:
model_name = 'AlexNet_v2_Trial_3.h5'

In [8]:
#Fit Model!
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=10)
mc = ModelCheckpoint('AlexNet_v2_Trial_3.h5', monitor='val_accuracy', mode='max', verbose=2, save_best_only=True)
history = cnn_model.fit(train_x, train_y, epochs = 10, batch_size = 32, validation_split = 0.1, verbose = 1, callbacks = [es, mc])

Train on 162 samples, validate on 18 samples
Epoch 1/10
160/162 [============================>.] - ETA: 0s - loss: 4.3935 - accuracy: 0.5000
Epoch 00001: val_accuracy improved from -inf to 0.44444, saving model to AlexNet_v3_Trial_3.h5
162/162 [==============================] - 14s 84ms/sample - loss: 4.4259 - accuracy: 0.5000 - val_loss: 1.7982 - val_accuracy: 0.4444
Epoch 2/10
160/162 [============================>.] - ETA: 0s - loss: 3.0101 - accuracy: 0.6562
Epoch 00002: val_accuracy did not improve from 0.44444
162/162 [==============================] - 9s 53ms/sample - loss: 2.9729 - accuracy: 0.6605 - val_loss: 7.3015 - val_accuracy: 0.4444
Epoch 3/10
160/162 [============================>.] - ETA: 0s - loss: 3.4451 - accuracy: 0.6875
Epoch 00003: val_accuracy did not improve from 0.44444
162/162 [==============================] - 10s 65ms/sample - loss: 3.4026 - accuracy: 0.6914 - val_loss: 7.7258 - val_accuracy: 0.4444
Epoch 4/10
160/162 [============================>.] - ETA:

In [17]:
#Evaluate Data
model = load_model('AlexNet_v2_Trial_1.h5')
loss, acc = model.evaluate(test_x, test_y, verbose = 0)
print("Test, Accuracy: ", acc)

Test, Accuracy:  0.51666665


In [11]:
# For 2048 - Incorrect
#Trial 1: 68.9% - 52%
#Trail 2: 82.2% - 51.6%
#Trail 3: 71.67% - 50%

# For 4096 - Different Validation and Test Split!
# Trail 1: 55.0% - 51.6%
# Trail 2: 55.6% - 67.2&
# Trail 3: 72.2% - 73%